In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import graph as gr
import utils as ut
import feature_selection as fs
import classification as cl
import os.path
import matplotlib.pyplot as plt
from pprint import pprint
import importlib
import community
import seaborn as sns
from gensim.models import KeyedVectors
from pyemd import emd
from gensim import corpora
from nltk.corpus import stopwords
from gensim.matutils import softcossim
import copy
import itertools
import pickle

Using Theano backend.
C:\Users\faisal\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Reshape
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [3]:
importlib.reload(ut)

<module 'utils' from 'C:\\Users\\faisal\\PycharmProjects\\linkPrediction\\utils.py'>

In [4]:
#word_vectors = KeyedVectors.load_word2vec_format('pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin', binary=True)

In [11]:
select_domain  = 0 #0:apnea ro 1:obesity
domain = ['apnea','obesity']
root = '../linkPrediction/'
filepath=[['dataset/apnea-all,3.csv',
         'dataset/apnea-distinct_keyword.csv'],
        ['dataset/obesity-all,3.csv',
         'dataset/obesity-distinct_keyword.csv']]
graphpath=root+'graphs/'+domain[select_domain]
datapath=root+'dataframes/'+domain[select_domain]
modelpath=root+'models/'+domain[select_domain]
column_split=['keyword','author_name','affiliation_1','affiliation_2','country']
#time=[parent_start_year, train_start_year, test_start_year, total_test_period_in_year, total_year_in_each_iteration]
time=[2007,2008,2015,1,1,200]
df, key_list = ut.load_dataset(filepath[select_domain], column_split)

In [12]:
graphpath

'../linkPrediction/graphs/apnea'

In [42]:
#dynamic train graphs and test graph build and save in disk 
importlib.reload(gr)
time=[1991,1992,1994,1,1,20]
gr.dynamic_train_test_graph_build(df, key_list, graphpath, time)

65
../linkPrediction/graphs/apnea\train_graph_1992.gpickle saved, nodes: 44 edges: 27
../linkPrediction/graphs/apnea\train_graph_1993.gpickle saved, nodes: 47 edges: 26
../linkPrediction/graphs/apnea\test_graph_1994.gpickle saved, nodes: 65 edges: 56
../linkPrediction/graphs/apnea\parent_graph_1991-1992.gpickle saved, nodes: 24 edges: 20
../linkPrediction/graphs/apnea\train_graph_1992-1994.gpickle saved, nodes: 65 edges: 51
../linkPrediction/graphs/apnea\test_graph_1994-1995.gpickle saved, nodes: 65 edges: 56


In [46]:
#load all graphs
time=[1991,1992,1994,1,1,20]
g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)

../linkPrediction/graphs/apnea\train_graph_1992.gpickle loaded, nodes: 44 edges: 27
../linkPrediction/graphs/apnea\train_graph_1993.gpickle loaded, nodes: 47 edges: 26
../linkPrediction/graphs/apnea\test_graph_1994.gpickle loaded, nodes: 65 edges: 56
../linkPrediction/graphs/apnea\parent_graph_1991-1992.gpickle loaded, nodes: 24 edges: 20
../linkPrediction/graphs/apnea\train_graph_1992-1994.gpickle loaded, nodes: 65 edges: 51
../linkPrediction/graphs/apnea\test_graph_1994-1995.gpickle loaded, nodes: 65 edges: 56


In [47]:
importlib.reload(fs)
importlib.reload(cl)

freq=10
#time=[2001,2007,2015,1,1,10]
train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                         g_test,
                                                                                                         g_parent,
                                                                                                         g_train_static, 
                                                                                                         g_test_static,  
                                                                                                         time, 
                                                                                                         freq)
node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, time)

pos
test_size: 45
neg
test_size: 1984
pos in time series: 44 neg in time series: 368 pos-neg ratio: 0.11956521739130435 total: 412


In [48]:
node_feature[1992]

,node_index,y_weight,term_art,term_aut,term_af1,term_af2,term_coun,closeness,degree,citation,degrees,node_type_aut,node_type_art,node_type_deg
0,12485,2,2.475175,3.053730,2.569181,2.112278,2.006527,0.134419,1,58,0.166667,20,20,20
1,6713,2,2.428600,2.675538,2.223101,1.483937,-0.497915,0.163925,2,622,0.333333,20,20,20
2,12542,2,4.089659,4.407155,3.763397,3.510583,3.163588,0.197674,6,1093,1.000000,20,20,20
3,12170,1,1.620439,1.946355,0.178916,0.195538,-0.198929,0.134419,1,84,0.166667,5,5,5
4,1127,2,-0.427348,-0.695082,-2.150384,-2.128324,-0.547052,0.000000,0,22,0.000000,3,3,3
5,3392,2,3.012551,2.113318,1.981689,1.765076,1.791026,0.160022,5,113,0.833333,5,5,5
6,6200,2,-0.427348,0.299742,-2.150384,-0.447278,-0.497915,0.000000,0,198,0.000000,20,20,20
7,6138,2,1.765809,0.750298,0.556947,1.470458,0.607043,0.094661,2,403,0.333333,20,1,20
8,12784,2,1.166037,1.026065,1.090874,2.372337,2.098449,0.117911,2,301,0.333333,5,5,5
9,6270,2,-0.427348,-1.092491,-2.150384,-1.887811,-0.497915,0.000000,0,204,0.000000,3,3,3


In [ ]:
print(len(train_data_static[(train_data_static['label']==1)]))
print(len(train_data_static[(train_data_static['label']==0)]))
print(len(g_train_static.nodes()))
print(len(g_train_static.edges()))
print(len(g_test_static.edges()))

In [ ]:
#save train_data and node feature
try:
    train_data
    filename= domain[select_domain]+"-train_data_"+str(time[1])+"-"+str(time[2])+".pkl"
    filename_path = os.path.join(datapath,filename)
    with open(filename_path,"wb") as f:
        pickle.dump(train_data,f)

    filename= domain[select_domain]+"-node_feature_"+str(time[1])+"-"+str(time[2])+".pkl"
    filename_path = os.path.join(datapath,filename)
    with open(filename_path,"wb") as f:
        pickle.dump(node_feature,f)
except NameError: 
    print('train data not exist')

In [9]:
# ###Extract from file
filename= domain[select_domain]+"-train_data_"+str(time[1])+"-"+str(time[2])+".pkl"
filename_path = os.path.join(datapath,filename)
with open(filename_path,"rb") as f:
    train_data = pickle.load(f)
    print('train data length:',len(train_data))
    
filename= domain[select_domain]+"-node_feature_"+str(time[1])+"-"+str(time[2])+".pkl"
filename_path = os.path.join(datapath,filename)
with open(filename_path,"rb") as f:
    node_feature = pickle.load(f)
    print('node feature length:',len(node_feature))

train data length: 7
node feature length: 7


In [ ]:
importlib.reload(fs)
importlib.reload(cl)

ts = time[1]
te = time[2]

row = train_data[ts].columns.values.tolist()
feature_names = row[2:len(row)]
all_combinations = itertools.chain(*[itertools.combinations(feature_names, i + 1)
                                     for i, _ in enumerate(feature_names)])
feature_list = list(all_combinations)
#selective_features =  set(['typeaut','typeart','typenode','y_weight1','res_aloc','cm','close','pref'])
selective_features =  set(['y_weight1','pref'])
domain_info =  domain[select_domain]+"-"+str(ts)+"-"+str(te)+"-"
models  = []
results = []
con = [20,64]
batch_size = con[1]
for feature in feature_list:
    columns_drop = list(set(feature_names).difference(set(feature)))
    if len(feature)==1 and feature[0] in selective_features:
        model_name = '+'.join(feature)
        model_name = domain_info+model_name
        pruned_data = copy.deepcopy(train_data)
        pruned_data = fs.drop_feature_columns(pruned_data, columns_drop, time)
        #data reshape for classification model
        X,y, data_len_dm = cl.reshape_feature_data_for_classification(pruned_data, edge_list, time)
        #training and testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
        model = cl.classification_model(X_train, X_test, y_train, y_test, data_len_dm, con, model_name)
        results.append(cl.model_evaluate(model, X_test, y_test, batch_size, model_name))
        #model save
        model_file_path = os.path.join(modelpath, model_name + ".h5")
        model.save(model_file_path)
        models.append(model_name)
        print(model_file_path)
        #x data save
        data_file_path = os.path.join(datapath,model_name+"-X"+".pkl")
        with open(data_file_path,"wb") as f:
            pickle.dump(X,f)
        print(data_file_path)
        #y data save
        data_file_path = os.path.join(datapath,model_name+"-y"+".pkl")
        with open(data_file_path,"wb") as f:
            pickle.dump(y,f)
        print(data_file_path)
#save result
name = domain[select_domain]+"-results_"+str(time[1])+"-"+str(time[2])+".pkl"
result_path = os.path.join(datapath,name)
ut.save_data(result_path,results)

In [ ]:
######training from loaded data
importlib.reload(fs)
importlib.reload(cl)
importlib.reload(ut)

ts = time[1]
te = time[2]
models  = []
results = []
con = [2,64]
batch_size = con[1]
#selective_features =  set(['typeaut','typeart','typenode','y_weight1','res_aloc','cm','close','pref'])
selective_features =  set(['y_weight1','pref'])
domain_info =  domain[select_domain]+"-"+str(ts)+"-"+str(te)+"-"
for feature in selective_features:
    model_name = domain_info+feature
    X_file_path = os.path.join(datapath,model_name+"-X"+".pkl")
    with open(X_file_path,"rb") as f:
        X = pickle.load(f)
    y_file_path = os.path.join(datapath,model_name+"-y"+".pkl")
    with open(y_file_path,"rb") as f:
        y = pickle.load(f)
    #data shape
    input_length = X.shape[1]
    input_dim = X.shape[2]
    if len(y.shape) == 1:
        output_dim = 1
    else:
        output_dim = len(y[0])
    data_len_dm = [input_length, input_dim, output_dim]
    ####training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 4)
    model = cl.classification_model(X_train, X_test, y_train, y_test, data_len_dm, con, model_name)
    results.append(cl.model_evaluate(model, X_test, y_test, batch_size, model_name))
    ####model save
    model_file_path = os.path.join(modelpath, model_name + ".h5")
    model.save(model_file_path)
    models.append(model_name)
    print(model_file_path)
#save result
name = domain[select_domain]+"-results_"+str(time[1])+"-"+str(time[2])+".pkl"
result_path = os.path.join(datapath,name)
ut.save_data(result_path,results)

In [ ]:
importlib.reload(ut)
###apnea result
domain_id =0 
resultpath=root+'dataframes/'+domain[domain_id]
name = domain[domain_id]+"-results_"+str(time[1])+"-"+str(time[2])+".pkl"
result_path = os.path.join(resultpath,name)
results_apnea = ut.load_data(result_path)
###obesity result
domain_id =1 
resultpath=root+'dataframes/'+domain[domain_id]
name = domain[domain_id]+"-results_"+str(time[1])+"-"+str(time[2])+".pkl"
result_path = os.path.join(resultpath,name)
results_obesity = ut.load_data(result_path)

In [ ]:
print(results_apnea[0]['model name'],results_apnea[0]['auc'],results_apnea[1]['auc'])
print(results_obesity[0]['model name'],results_obesity[0]['auc'],results_obesity[1]['auc'])

In [ ]:
names = {'typeaut':'T-Aut',
         'typeart':'T-Art','typenode':'T-Key','y_weight1':'T-Key-y','res_aloc':'RA','cm':'CM','close':'CC','pref':'PA'}
colors = ['g','r','c','m','y','k','C0','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
xtick=[]
fig, ax = plt.subplots(figsize=(10,6)) 
#fig = plt.figure(figsize=(10,6))
ax.set_axisbelow(True)
ax.grid(axis='y', linestyle='--')
plt.ylim(.3, 1)
bar_width = 0.35
result_lat = {'apnea':{},'obesity':{}}
for i in range(len(results_apnea)):
    name = results_apnea[i]['model name'].split('-')[3]
    result_lat['apnea'][names[name]]=results_apnea[i]['auc']
    result_lat['obesity'][names[name]]=results_obesity[i]['auc']
    ax.bar(i,results_obesity[i]['auc'] ,bar_width,align='center',alpha=0.8,color = colors[i])
    ax.text(i, .5,r'$D_{obs}$', color='k',horizontalalignment='center',rotation=90)
    ax.bar(i+bar_width+.06,results_apnea[i]['auc'] ,bar_width,align='center', alpha=0.4,color = colors[i])
    ax.text(i+bar_width+.06, .5, r'$D_{sas}$', color='k',horizontalalignment='center',rotation=90)
    xtick.append(names[name])
x_pos = np.arange(len(xtick))
plt.xticks(x_pos+.2, xtick, rotation=0)
plt.xlabel('Feature name', fontsize=12)
plt.ylabel('AUC', fontsize=12)
###save
figpath=root+'dataframes/'
fig_file_name = os.path.join(figpath,'auc'+ ".png")
plt.savefig(fig_file_name)
###show
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
for i,row in enumerate(results_apnea):
    if i==1 or i==2 or i==5 or i==7:
        name = row['model name'].split('-')[3]
        plt.plot(row['recall'],row['precision'],label = names[name])
plt.legend(loc='best')
plt.title("PR curve $(D_{sas})$", position=(0.4, 0.9),fontsize=12)
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.tight_layout()
plt.subplot(2,2,2)
for i,row in enumerate(results_apnea):
    if i==1 or i==2 or i==5 or i==7:
        name = row['model name'].split('-')[3]
        plt.plot(row['false positive'],row['true positive'],label = names[name])
plt.legend(loc='best')
plt.title("ROC curve $(D_{sas})$", position=(0.4, 0.9),fontsize=12)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.tight_layout()

plt.subplot(2,2,3)
for i,row in enumerate(results_obesity):
    if i==1 or i==2 or i==5 or i==7:
        name = row['model name'].split('-')[3]
        plt.plot(row['recall'],row['precision'],label = names[name])
plt.legend(loc='best')
plt.title("PR curve $(D_{obs})$", position=(0.4, 0.9),fontsize=12)
plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.tight_layout()
plt.subplot(2,2,4)
for i,row in enumerate(results_obesity):
    if i==1 or i==2 or i==5 or i==7:
        name = row['model name'].split('-')[3]
        plt.plot(row['false positive'],row['true positive'],label = names[name])
plt.legend(loc='best')
plt.title("ROC curve $(D_{obs})$", position=(0.4, 0.9),fontsize=12)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.tight_layout()
###save
figpath=root+'dataframes/'
fig_file_name = os.path.join(figpath,'pr-roc'+ ".png")
plt.savefig(fig_file_name)
#show
plt.show()

In [ ]:
result_lat

In [ ]:
result_df = pd.DataFrame(result_lat)

In [ ]:
result_df

In [ ]:
print(result_df.to_latex(float_format=lambda x: '%.2f' % x))

In [36]:
#nx.write_gexf(g_train[2008], "test.gexf")

In [10]:
#gephi graph
ts = time[1]
te = time[2]
label = dict(zip(key_list['id'], key_list['keyword']))
for t in range(ts,te):
    gephy_g = nx.Graph()
    nodes = list(g_train[t].nodes())
    edges = list(g_train[t].edges())
    gephy_g.add_nodes_from(nodes,year=t)
    gephy_g.add_edges_from(edges,year=t)
    aut = dict(zip(node_feature[t]['node_index'], node_feature[t]['term_aut']))
    types = dict(zip(node_feature[t]['node_index'], node_feature[t]['node_type_aut']))
    nx.set_node_attributes(gephy_g, aut, 'aut')
    nx.set_node_attributes(gephy_g, label, 'label')
    nx.set_node_attributes(gephy_g, types, 'types')
    file = "gephy\g-obesity-"+str(t)+".gexf"
    nx.write_gexf(gephy_g, file)

In [31]:
g_train[2008].nodes[5684]

{'art_id': {96834,
  97226,
  110362,
  110380,
  110449,
  110601,
  110678,
  110750,
  110853,
  111052,
  111528,
  124496,
  124983},
 'citation': 275,
 'year': {2008},
 'title': {'Breathing disorders in congestive heart failure Gender etiology and mortality',
  'Daytime sleepiness relates to snoring independent of the apnea-hypopnea index in women from the general population',
  'Gender and age differences in polysomnography findings and sleep complaints of patients referred to a sleep laboratory',
  'Gender differences in Saudi patients with obstructive sleep apnea',
  'Gender differences in polysomnographic findings in Turkish patients with obstructive sleep apnea syndrome',
  'Health and health-related quality of life differences between men and women who seek gastric bypass surgery',
  'Obstructive sleep apnoea syndrome OSAS in women A specific entity Pathologie pulmonaire au fminin Le SAOS de la femme une entit particulire',
  'Rapid eye movement-related sleep-disordered bre